<a href="https://colab.research.google.com/github/dagousket/ML-course-VIB-2020/blob/master/Histone_Marks_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Let's go Deep Learning style!


In [100]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import PIL
import tensorflow as tf

random_seed = 666
np.random.seed(random_seed)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import log_loss, accuracy_score

### Import and format data

In [101]:
train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")

# Save labels in external variable
train_labels = train.pop("Label")
train_index_col = train.pop("GeneId")
test_index_col = test.pop("GeneId")

In [102]:
marks = list(set([c_name.split("_")[0] for c_name in train.columns]))

# Shape of array will be 100 bins * 5 histone marks

def pool_marks(mybins, marks) :
  df = pd.DataFrame()
  for mark in marks :
      df.reset_index()
      df[mark] = mybins[[mark + "_" + str(bin) for bin in range(0,100)]].to_numpy()
  df = df.to_numpy() / 200
  return df

def get_input(X, marks):
  res_out = X.apply(lambda y : pool_marks(y, marks), axis = 1)
  return pd.DataFrame(res_out)

X = [A for A in get_input(train, marks).iloc[:,0]]
X = np.array([np.array(dp) for dp in X])

T = [A for A in get_input(test, marks).iloc[:,0]]
T = np.array([np.array(dp) for dp in T])

In [103]:
print(X.shape)
print(T.shape)
X[:5]

(10436, 100, 5)
(5049, 100, 5)


array([[[0.015, 0.01 , 0.005, 0.005, 0.005],
        [0.005, 0.025, 0.005, 0.01 , 0.   ],
        [0.015, 0.04 , 0.005, 0.015, 0.025],
        ...,
        [0.015, 0.08 , 0.01 , 0.04 , 0.03 ],
        [0.01 , 0.075, 0.01 , 0.055, 0.025],
        [0.01 , 0.06 , 0.01 , 0.05 , 0.01 ]],

       [[0.   , 0.005, 0.   , 0.   , 0.005],
        [0.01 , 0.01 , 0.01 , 0.   , 0.   ],
        [0.005, 0.005, 0.01 , 0.   , 0.   ],
        ...,
        [0.   , 0.005, 0.   , 0.005, 0.   ],
        [0.   , 0.   , 0.005, 0.005, 0.   ],
        [0.   , 0.   , 0.005, 0.005, 0.   ]],

       [[0.005, 0.005, 0.035, 0.015, 0.005],
        [0.01 , 0.005, 0.02 , 0.01 , 0.005],
        [0.   , 0.02 , 0.01 , 0.01 , 0.005],
        ...,
        [0.005, 0.005, 0.01 , 0.   , 0.005],
        [0.   , 0.   , 0.005, 0.005, 0.01 ],
        [0.   , 0.005, 0.005, 0.005, 0.005]],

       [[0.015, 0.02 , 0.02 , 0.01 , 0.005],
        [0.015, 0.005, 0.   , 0.01 , 0.005],
        [0.005, 0.005, 0.005, 0.03 , 0.005],
        ..

In [106]:
#one-hot encode target column
from keras.utils import to_categorical
y_train = to_categorical(train_labels)
y_train = np.array([Y for Y in y_train])
print(y_train.shape)
y_train[:5]

(10436,)


array([0., 0., 1., 1., 0.], dtype=float32)

In [107]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_train, test_size=0.1)

### Make a DL model

In [108]:
from keras.layers.recurrent import LSTM

num_classes = 2

model = Sequential([
  layers.InputLayer(input_shape=(100,5)),
  layers.Conv1D(30, kernel_size=3, padding="valid", activation='relu', kernel_initializer='random_uniform', data_format ="channels_last"),
  layers.MaxPooling1D(pool_size=3, strides=1, padding='valid'),
  layers.Dropout(0.2),
  layers.TimeDistributed(layers.Dense(128, activation='relu')),
  layers.Bidirectional(LSTM(128, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(20, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(num_classes, activation='softmax')])



model.compile(optimizer='adam',
              loss=tf.compat.v1.losses.log_loss,
              metrics=['accuracy'])

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 98, 30)            480       
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 96, 30)            0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 96, 30)            0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 96, 128)           3968      
_________________________________________________________________
bidirectional_6 (Bidirection (None, 96, 256)           263168    
_________________________________________________________________
dropout_29 (Dropout)         (None, 96, 256)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 24576)           

In [110]:

history=model.fit(X_train, y_train, batch_size=500 , epochs=3 , verbose=1, validation_data=(X_test, y_test))


Epoch 1/3
19/19 [==============================] - 13s 667ms/step - loss: 8.0323 - accuracy: 0.4962 - val_loss: 7.9101 - val_accuracy: 0.5038
Epoch 2/3
19/19 [==============================] - 12s 655ms/step - loss: 8.0323 - accuracy: 0.4962 - val_loss: 7.9101 - val_accuracy: 0.5038
Epoch 3/3
19/19 [==============================] - 12s 654ms/step - loss: 8.0323 - accuracy: 0.4962 - val_loss: 7.9101 - val_accuracy: 0.5038


In [95]:
from sklearn.metrics import log_loss, accuracy_score

predictions_train_prob = model.predict(X_train)
predictions_val_prob = model.predict(T)


In [96]:
print(predictions_train_prob[:10])
print(y_train[:10])

#H = [X[0] for X in predictions_train_prob]

#print("Log-loss: (%f) %f"%(log_loss(y_train,predictions_train_prob[:,1]),log_loss(y_test,predictions_val_prob[:,1])))

[[0.98182136 0.0181787 ]
 [0.95644    0.04355998]
 [0.9664179  0.03358207]
 [0.5074502  0.4925498 ]
 [0.5071384  0.49286163]
 [0.9244579  0.07554207]
 [0.94158614 0.05841379]
 [0.55895877 0.44104123]
 [0.31481078 0.68518925]
 [0.9720638  0.02793625]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [97]:
# Predict Kaggle set
H = [X[1] for X in predictions_val_prob]
print(H[:5])
print(len(H))
print(len(test_index_col))

predictions_df = pd.DataFrame({"GeneId":test_index_col,"Label":H})
predictions_df.head()
predictions_df.to_csv('submission_dl3.csv', index=False)

[0.9453295, 0.11799873, 0.056134973, 0.8856515, 0.83883196]
5049
5049
